<a href="https://colab.research.google.com/github/Gamalielmch/CNN_PDI/blob/main/confusion_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, precision_recall_fscore_support
import cv2
import pandas as pd

In [19]:
# load dataset
(trainX, trainY), (testX, testY) = mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))



In [20]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [21]:
# convert from integers to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')
# normalize to range 0-1
trainX = trainX / 255.0
testX = testX / 255.0

In [22]:
# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#     model.summary()
    return model

In [23]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        print(trainX.shape)
        history = model.fit(trainX, trainY, epochs=1, batch_size=32, validation_data=(testX, testY), verbose=1)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
    return scores, histories, model

In [24]:
def summarize_diagnostics(histories):
    fig, (ax1, ax2) = pyplot.subplots(2, 1)
    for i in range(len(histories)):
        # plot loss
        ax1.title.set_text('Cross Entropy Loss')
        ax1.plot(histories[i].history['loss'], color='blue', label='train')
        ax1.plot(histories[i].history['val_loss'], color='orange', label='test')
        # plot accuracy
#         pyplot.subplot(2, 1, 2)
        ax2.title.set_text('Classification Accuracy')
        ax2.plot(histories[i].history['accuracy'], color='blue', label='train')
        ax2.plot(histories[i].history['val_accuracy'], color='orange', label='test')
    pyplot.show()

In [25]:
# summarize model performance
def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
    # box and whisker plots of results
    pyplot.boxplot(scores)
    pyplot.show()

In [26]:
scores, histories, model = evaluate_model(trainX, trainY)


(48000, 28, 28, 1)
1500/1500 [==============================] - 26s 17ms/step - loss: 0.1968 - accuracy: 0.9399 - val_loss: 0.0956 - val_accuracy: 0.9722
> 97.225
(48000, 28, 28, 1)
1500/1500 [==============================] - 26s 17ms/step - loss: 0.1834 - accuracy: 0.9444 - val_loss: 0.0942 - val_accuracy: 0.9732
> 97.317
(48000, 28, 28, 1)
1500/1500 [==============================] - 26s 17ms/step - loss: 0.1901 - accuracy: 0.9425 - val_loss: 0.0899 - val_accuracy: 0.9729
> 97.292
(48000, 28, 28, 1)
1500/1500 [==============================] - 26s 17ms/step - loss: 0.1915 - accuracy: 0.9419 - val_loss: 0.0772 - val_accuracy: 0.9774
> 97.742
(48000, 28, 28, 1)
1500/1500 [==============================] - 26s 17ms/step - loss: 0.1864 - accuracy: 0.9431 - val_loss: 0.0850 - val_accuracy: 0.9758
> 97.583


In [28]:
y_tar=np.argmax(testY,1)
pY_test=model.predict(testX)
predections= pY_test.argmax(axis=1)
cm=confusion_matrix(y_tar,predections)
print(cm)
model.save('my_model.h5')

[[ 964    0    1    0    0    4    5    1    4    1]
 [   0 1121    2    1    0    1    5    0    5    0]
 [   2    5 1002    1    2    0    0   10   10    0]
 [   0    0    4  979    0    9    0    5   13    0]
 [   0    0    3    0  965    0    5    1    2    6]
 [   1    0    0    2    0  883    3    0    3    0]
 [   6    3    1    0    3    3  935    1    6    0]
 [   0    4   10    4    0    2    0 1002    4    2]
 [   1    0    3    3    2    2    0    3  959    1]
 [   4    5    1    2   12    5    1   10    5  964]]
